In [1]:
from deepface import DeepFace
import cv2
import time
import numpy as np
from tqdm import tqdm

In [2]:
FILE=  "./snap.png"

In [15]:
img = cv2.imread(FILE)
TARGET_FRAME_H = 480
h, w, _  = img.shape
red = cv2.resize(img, (TARGET_FRAME_H, int(TARGET_FRAME_H*h/w)))

In [5]:
backends = ["opencv","ssd", "retinaface", "mtcnn", "dlib"]

In [11]:
def get_time(backend):
    times = []
    for _ in range(100):
        s = time.time()
        _ = DeepFace.detectFace(red, detector_backend=backend)
        e = time.time()
        times.append(e-s)
    return np.mean(times), np.std(times)

In [12]:
results = [(b, *get_time(b)) for b in tqdm(backends)]




  0%|          | 0/5 [00:47<?, ?it/s]



100%|██████████| 5/5 [04:23<00:00, 52.66s/it]


In [13]:
results

[('opencv', 0.07655892133712769, 0.0028219315573466664),
 ('ssd', 0.04864974021911621, 0.005127733076269846),
 ('retinaface', 1.6307357454299927, 0.2659187709625731),
 ('mtcnn', 0.44707278728485106, 0.043126353276591665),
 ('dlib', 0.42969278573989866, 0.03132568673026541)]

# OpenCV

In [14]:
import cv2

In [16]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
ocv_time = []
for _ in tqdm(range(100)):
    s = time.time()
    face = face_cascade.detectMultiScale(img, 1.1, 4)
    e = time.time()
    ocv_time.append(e-s)
print(f"opencv - mean: {np.mean(ocv_time)}, std: {np.std(ocv_time)}")

# Dlib

In [5]:
import dlib

In [51]:
face_detector = dlib.get_frontal_face_detector()
img = cv2.imread(FILE)

In [69]:
dlib_times = []
TARGET_FRAME_H = 480
h, w, _  = img.shape
red = cv2.resize(img, (TARGET_FRAME_H, int(TARGET_FRAME_H*h/w)))
for _ in tqdm(range(100)):
    s = time.time()
    _ = face_detector(red, 1)
    e = time.time()
    dlib_times.append(e-s)
dlib_mean = np.mean(dlib_times)
dlib_std = np.std(dlib_times)

100%|██████████| 100/100 [00:07<00:00, 14.15it/s]


In [70]:
dlib_mean ,dlib_std

(0.07036459922790528, 0.005954892001096505)

In [55]:
cv2.imwrite("test.png", img)

True

In [71]:
ssd_times = []
for _ in tqdm(range(100)):
    s = time.time()
    _ = DeepFace.analyze(img, actions = ['emotion'])
    e = time.time()
    ssd_times.append(e-s)
ssd_mean = np.mean(ssd_times)
ssd_std = np.std(ssd_times)


100%|██████████| 100/100 [00:08<00:00, 12.09it/s]


In [57]:
(ssd_mean, ssd_std)

(0.04652724504470825, 0.009975967952996665)

In [72]:
find_times = []
for _ in tqdm(range(100)):
    s = time.time()
    find = DeepFace.detectFace(img, detector_backend="ssd", align=False)
    e = time.time()
    find_times.append(e-s)
find_mean = np.mean(find_times)
find_std = np.std(find_times)

100%|██████████| 100/100 [00:02<00:00, 40.10it/s]


In [73]:
(find_mean, find_std)

(0.024827897548675537, 0.003196199845505427)

In [74]:
cv2.imwrite("find.jpg", find*255)

True

# PAZ

In [75]:
from EmotionProcessor import EmotionProcessor

In [76]:
ep = EmotionProcessor()

In [81]:
def paz_pipeline(backend):
    paz_times = []
    for _ in tqdm(range(20)):
        s = time.time()
        find = DeepFace.detectFace(img, detector_backend=backend, align=False)
        #find_rgb = cv2.cvtColor(find*255, cv2.COLOR_BGR2RGB)
        _ = ep(find*255)
        e = time.time()
        paz_times.append(e-s)
    paz_mean = np.mean(paz_times)
    paz_std = np.std(paz_times)
    return paz_mean, paz_std

In [82]:
results_paz = [(b, *paz_pipeline(b)) for b in backends]

100%|██████████| 20/20 [00:00<00:00, 20.13it/s]


In [83]:
results_paz

[('opencv', 0.07658860683441163, 0.0038083545801218407),
 ('dlib', 0.5125818610191345, 0.07607236507055605),
 ('retinaface', 1.7534725785255432, 0.6074308977522134),
 ('ssd', 0.04941911697387695, 0.002139623092962623)]

# Eye contact

In [85]:
from EyeContactDetector import EyeContactDetector

In [86]:
ec = EyeContactDetector()

loading saved model weights


In [87]:
def eye_pipeline(backend):
    paz_times = []
    for _ in tqdm(range(20)):
        s = time.time()
        find = DeepFace.detectFace(img, detector_backend=backend, align=False)
        #find_rgb = cv2.cvtColor(find*255, cv2.COLOR_BGR2RGB)
        _ = ec.detect_eye_contact_on_face(find*255)
        e = time.time()
        paz_times.append(e-s)
    paz_mean = np.mean(paz_times)
    paz_std = np.std(paz_times)
    return paz_mean, paz_std

In [88]:
results_eye = [(b, *paz_pipeline(b)) for b in backends]

100%|██████████| 20/20 [00:00<00:00, 20.74it/s]


In [89]:
results_eye

[('opencv', 0.08226438760757446, 0.010470864126201565),
 ('dlib', 0.4861815690994263, 0.012266820965071461),
 ('retinaface', 1.6252570152282715, 0.05895252293748639),
 ('ssd', 0.048024463653564456, 0.004167891071739218)]

In [96]:
def run_on_video(backend):
    VID = "test_face.mov"
    cap = cv2.VideoCapture(VID)
    ret = True
    missing_frames = 0
    frames = 0
    while ret and frames < 100:
        ret, frame = cap.read()
        if frame is None:
            break
        try:
            find = DeepFace.detectFace(frame, detector_backend=backend, align=False)
            #find_rgb = cv2.cvtColor(find*255, cv2.COLOR_BGR2RGB)
            _ = ep(find*255)
            print("detected")
        except:
            print("face not found")
            missing_frames += 1 
        frames += 1
    return missing_frames
    

In [97]:
frames = [(b, run_on_video(b)) for b in backends]

detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
detected
face not found
detected
face not found
detected
face not found
face not found
detected
face not found
face not found
face not found
face not found
face not found
face not found
detected
detected
detected
detected
dete

In [98]:
frames

[('opencv', 10), ('ssd', 0)]